In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 10)
!date

%load_ext autoreload
%autoreload 2

Mon Jun 10 12:17:27 PDT 2019


# Load IPUMS 1940 Census Microdata, and filter to just WA state

Codebook: https://usa.ipums.org/usa/resources/1940CensusDASTestData/EXT1940USCB.cbk

My previous approach failed, because the person rows do not include the location details.  Annoying!  How to address?  First make a household file, then make a person file, getting the location details from the household file.

In [2]:
with open('data/full_ipums_1940_census_colspecs.txt') as f:
    colspec_str = f.read()

In [3]:
colspecs = pd.Series()
for line in colspec_str.split('\n'):
    name = line.split()[0].strip()
    val_str = line.split()[2]
    
    start_str = val_str.split('-')[0]
    start_index = int(start_str) - 1

    end_str = val_str.split('-')[-1]
    end_index = int(end_str)
    
#     print(name, val_str, (start_index, end_index))
    colspecs[name] = (start_index, end_index)

# Pass 1: Generate HH file for WA state

In [ ]:
%%time

fname = '/home/j/Project/Models/us_census/EXT1940USCB.dat'

iter = pd.read_fwf(fname, list(colspecs), header=None, names=colspecs.index,
                   chunksize=1_000_000)
df_list = []

for i, df_i in enumerate(iter):
    print(f'Processing chunk {i}, a dataframe with shape {df_i.shape}')
    df_list.append(
        df_i[df_i.RECTYPE == 'H'].filter(
            ['RECTYPE', 'SERIAL', 'NUMPREC', 'STATEFIP',
             'COUNTY', 'ENUMDIST', 'GQ',]) # 'AGE', 'HISPAN', 'RACE'])
    )
#     if i > 2: break

Processing chunk 0, a dataframe with shape (1000000, 155)
Processing chunk 1, a dataframe with shape (1000000, 155)
Processing chunk 2, a dataframe with shape (1000000, 155)
Processing chunk 3, a dataframe with shape (1000000, 155)
Processing chunk 4, a dataframe with shape (1000000, 155)
Processing chunk 5, a dataframe with shape (1000000, 155)
Processing chunk 6, a dataframe with shape (1000000, 155)
Processing chunk 7, a dataframe with shape (1000000, 155)
Processing chunk 8, a dataframe with shape (1000000, 155)
Processing chunk 9, a dataframe with shape (1000000, 155)
Processing chunk 10, a dataframe with shape (1000000, 155)
Processing chunk 11, a dataframe with shape (1000000, 155)
Processing chunk 12, a dataframe with shape (1000000, 155)
Processing chunk 13, a dataframe with shape (1000000, 155)
Processing chunk 14, a dataframe with shape (1000000, 155)
Processing chunk 15, a dataframe with shape (1000000, 155)
Processing chunk 16, a dataframe with shape (1000000, 155)
Process

In [ ]:
df = pd.concat(df_list)
print(f'Loaded data for one state, a dataframe with shape {df.shape}')

In [ ]:
# save relevant rows for comparison
df.to_csv('/home/j/Project/Models/us_census/hh_1940_raw.csv')

In [12]:
!ls -halt /home/j/Project/Models/us_census/hh_1940_raw.csv

-rw-r--r-- 1 abie Domain Users 1.3G Jun 10 22:43 /home/j/Project/Models/us_census/hh_1940_raw.csv


In [13]:
df.shape

(38492963, 7)

In [14]:
df.STATEFIP.value_counts()

36    4153082
42    2732080
6     2473589
17    2432588
39    2057163
       ...   
50     102075
56      80146
10      78974
32      38878
2       24277
Name: STATEFIP, Length: 51, dtype: int64

In [15]:
df.RECTYPE.value_counts()

H    38492963
Name: RECTYPE, dtype: int64

In [16]:
df.GQ.value_counts()

1    36201432
6      863664
3      763242
4      530445
2      134180
Name: GQ, dtype: int64